In [1]:
import os
from PIL import Image

In [2]:
fd0 = "/home/jupyter/ai_font/data/exp0820/processed/train_whole/플레이브밤비"
fd1 = "/home/jupyter/ai_font/data/exp0820/report/over/플레이브밤비"
files1 = sorted([f for f in os.listdir(fd1) if f.endswith(".png")])
fd2 = "/home/jupyter/ai_font/data/exp0820/report/finetune"
files2 = sorted([f for f in os.listdir(fd2) if "." not in f])

In [4]:
fd2 = "/home/jupyter/ai_font/data/exp0820/report/finetune"

In [5]:
sorted([f for f in os.listdir(fd2) if "." not in f])

['i0',
 'i1000',
 'i10000',
 'i11000',
 'i12000',
 'i13000',
 'i14000',
 'i15000',
 'i16000',
 'i17000',
 'i18000',
 'i19000',
 'i2000',
 'i20000',
 'i21000',
 'i22000',
 'i23000',
 'i24000',
 'i25000',
 'i26000',
 'i27000',
 'i28000',
 'i29000',
 'i3000',
 'i30000',
 'i4000',
 'i5000',
 'i6000',
 'i7000',
 'i8000',
 'i9000']

In [6]:
import os
import copy
import numpy as np
import pandas as pd
from PIL import Image

class ImageConcat:
    def __init__(self, n_h, n_v, resolution=1000):
        self.n_h = n_h
        self.n_v = n_v
        self.r = resolution
        self.img = Image.new("L", (self.n_h*self.r, self.n_v*self.r), color=255)# color=(255,255,255))
        self.cursor = [0,0]
        self.max_h = 0
    
    def move_cursor_h(self, r):
        self.cursor[0] += r
        if self.cursor[0] > self.max_h:
            self.max_h = copy.deepcopy(self.cursor[0])
        
    def move_cursor_v(self):
        self.cursor[1] += self.r
        
    def reset_cursor_h(self):
        self.cursor[0] = 0
        
    def append_img(self, path, cut_margin=True, clip_white=False):
        try:
            img = Image.open(path).convert('L').resize((self.r,self.r), Image.BILINEAR)
            if cut_margin:
                img = self.cut_margin(img, clip_white=clip_white)
            self.img.paste(img, tuple(self.cursor))
            return img.size
        except:
            # print(f"Not Appended: {path}")
            return None
        
    def add_letter(self, path, cut_margin=True, clip_white=False):
        imgsize = self.append_img(path, cut_margin=cut_margin, clip_white=clip_white)
        if imgsize is None:
            self.move_cursor_h(self.r)
        else:
            self.move_cursor_h(imgsize[0])
    
    def add_space(self):
        self.move_cursor_h(int(self.r/3))
        
    def enter(self):
        self.move_cursor_v()
        self.reset_cursor_h()
        
    def cut_margin(self, img, clip_white):
        npimg = 255 - np.array(img)
        if clip_white:
            npimg[np.where(npimg<10)] = 0 
        wmin = npimg.sum(0).nonzero()[0].min()
        wmax = npimg.sum(0).nonzero()[0].max()
        # hmin = npimg.sum(1).nonzero()[0].min()
        # hmax = npimg.sum(1).nonzero()[0].max()

        npimg = 255 - npimg[:,wmin:wmax+1]
        img = Image.fromarray(npimg)
        return img


In [7]:
letters = np.unique([f[-7] for f in files1])

In [8]:
letters

array(['갊', '갸', '곬', '높', '뉑', '닠', '닦', '덤', '돛', '땍', '라', '럽', '롵',
       '많', '벚', '뽱', '쁘', '삯', '셴', '쉽', '쏟', '씩', '얹', '없', '왼', '읨',
       '잃', '쟨', '좋', '줬', '짚', '짧', '쫄', '쯷', '찧', '찾', '치', '캐', '탤',
       '턔', '핥', '훨'], dtype='<U1')

In [29]:
a = []
for l in letters:
    tetters = [f[-5] for f in files1 if f[-7] == l]
    a.append(len(tetters))

In [30]:
max(a)

21

In [32]:
img = ImageConcat(n_v=len(letters), n_h=max(a)*2,resolution=96)

In [34]:
for l in letters:
    img.add_letter(f"{fd0}/플레이브밤비__closing__{l}.png", cut_margin=False, clip_white=False)
    img.enter()
    tetters = [f[-5] for f in files1 if f[-7] == l]
    for t in tetters:
        img.add_letter(f"{fd1}/플레이브밤비__over__{l}_{t}.png", cut_margin=False, clip_white=False)
    img.enter()

In [36]:
img.img.save("inpainting_vis.png")

In [9]:
def get_all_korean():

    def nextKorLetterFrom(letter):
        lastLetterInt = 15572643
        if not letter:
            return '가'
        a = letter
        b = a.encode('utf8')
        c = int(b.hex(), 16)

        if c == lastLetterInt:
            return False

        d = hex(c + 1)
        e = bytearray.fromhex(d[2:])

        flag = True
        while flag:
            try:
                r = e.decode('utf-8')
                flag = False
            except UnicodeDecodeError:
                c = c+1
                d = hex(c)
                e = bytearray.fromhex(d[2:])
        return e.decode()

    returns = []
    flag = True
    k = ''
    while flag:
        k = nextKorLetterFrom(k)
        if k is False:
            flag = False
        else:
            returns.append(k)
    return returns

mapper = {}
for letter in get_all_korean():
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2
    mapper[(ch1, ch2, ch3)] = letter

def target_letters1(letter, mapper):
    
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2

    targets = [mapper[(l1,ch2,ch3)] for l1 in range(19)] 
    return targets

def target_letters2(letter, mapper):
    hlist = [0,1,2,3,4,5,6,7,20] # ㅏㅐㅑㅒㅓㅔㅕㅖㅣ
    vlist = [8,12,13,17,18] # ㅗㅛㅜㅠㅡ
    clist = [9,10,11,14,15,16,19] # ㅘㅙㅚㅝㅞㅟㅢ
    
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2

    # if ch2 in hlist:
    #     targets = [mapper[(ch1,l2,ch3)] for l2 in hlist] 
    # elif ch2 in vlist:
    #     targets = [mapper[(ch1,l2,ch3)] for l2 in vlist] 
    # else:
    #     targets = [mapper[(ch1,l2,ch3)] for l2 in clist] 
    # return targets
    targets = [mapper[(ch1,l2,ch3)] for l2 in range(21)] 
    return targets
    

def target_letters3(letter, mapper):
    hlist = [0,1,2,3,4,5,6,7,20] # ㅏㅐㅑㅒㅓㅔㅕㅖㅣ
    vlist = [8,12,13,17,18] # ㅗㅛㅜㅠㅡ
    clist = [9,10,11,14,15,16,19] # ㅘㅙㅚㅝㅞㅟㅢ
    
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2

    targets = [mapper[(ch1,ch2,l3)] for l3 in range(28)] 
    return targets


In [11]:
files2 = [f"i{str(i*1000)}" for i in range(20,31)]

In [12]:
for exp in files2:
    img = ImageConcat(n_v=len(letters)*4, n_h=20, resolution=96)
    for l in letters:
        img.add_letter(f"{fd0}/플레이브밤비__closing__{l}.png", cut_margin=False, clip_white=False)
        img.enter()
        tetters = target_letters1(l, mapper)
        for i, t in enumerate(tetters):
            tath = f"{fd2}/{exp}/플레이브밤비__{t}.png"
            if os.path.exists(tath):
                img.add_letter(tath, cut_margin=False, clip_white=False)
            if i >= 20:
                break
        img.enter()
        tetters = target_letters2(l, mapper)
        for i, t in enumerate(tetters):
            tath = f"{fd2}/{exp}/플레이브밤비__{t}.png"
            if os.path.exists(tath):
                img.add_letter(tath, cut_margin=False, clip_white=False)
            if i >= 20:
                break
        img.enter()
        tetters = target_letters3(l, mapper)
        for i, t in enumerate(tetters):
            tath = f"{fd2}/{exp}/플레이브밤비__{t}.png"
            if os.path.exists(tath):
                img.add_letter(tath, cut_margin=False, clip_white=False)
            if i >= 20:
                break
        img.enter()
    img.img.save(f"finetune_{exp}.png")

In [54]:
os.listdir(fd2)

['i3000',
 '.ipynb_checkpoints',
 'i14000',
 'i6000',
 'i16000',
 'i1000',
 'i11000',
 'i17000',
 'i2000',
 'i9000',
 'i0',
 'i7000',
 'Untitled.ipynb',
 'i12000',
 'i13000',
 'i15000',
 'i4000',
 'i10000',
 'i5000',
 'i8000',
 'i18000']